# Cleaning an Orders Dataset with PySpark

![ecommerce_analytics-1224x532](ecommerce_analytics-1224x532.jpg)


As a Data Engineer at an electronics e-commerce company, Voltmart, you have been requested by a peer Machine Learning team to clean the data containing the information about orders made last year. They are planning to further use this cleaned data to build a demand forecasting model. To achieve this, they have shared their requirements regarding the desired output table format.

An analyst shared a parquet file called `"orders_data.parquet"` for you to clean and preprocess. 

You can see the dataset schema below along with the **cleaning requirements**:

## `orders_data.parquet`

| column | data type | description | cleaning requirements | 
|--------|-----------|-------------|-----------------------|
| `order_date` | `timestamp` | Date and time when the order was made | _Modify: Remove orders placed between 12am and 5am (inclusive); convert from timestamp to date_ |
| `time_of_day` | `string` | Period of the day when the order was made | _New column containing (lower bound inclusive, upper bound exclusive): "morning" for orders placed 5-12am, "afternoon" for orders placed 12-6pm, and "evening" for 6-12pm_ |
| `order_id` | `long` | Order ID | _N/A_ |
| `product` | `string` | Name of a product ordered | _Remove rows containing "TV" as the company has stopped selling this product; ensure all values are lowercase_ |
| `product_id` | `double` | Product ID | _N/A_ |
| `category` | `string` | Broader category of a product | _Ensure all values are lowercase_ |
| `purchase_address` | `string` | Address line where the order was made ("House Street, City, State Zipcode") | _N/A_ |
| `purchase_state` | `string` | US State of the purchase address | _New column containing: the State that the purchase was ordered from_ |
| `quantity_ordered` | `long` | Number of product units ordered | _N/A_ |
| `price_each` | `double` | Price of a product unit | _N/A_ |
| `cost_price` | `double` | Cost of production per product unit | _N/A_ |
| `turnover` | `double` | Total amount paid for a product (quantity x price) | _N/A_ |
| `margin` | `double` | Profit made by selling a product (turnover - cost) | _N/A_ |

<br>

# Initialize Spark Session

In [1]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

spark = (
    SparkSession
    .builder
    .appName('cleaning_orders_dataset_with_pyspark')
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/10/13 16:31:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Read data source

In [2]:
orders_data = spark.read.parquet('orders_data.parquet')
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950


# Remove unneccessary order date

In [3]:
orders_data = orders_data \
    .filter("HOUR(order_date) NOT BETWEEN 1 AND 4") \
    .filter("product NOT LIKE '%TV%'")

# Transform order_date and time_of_day

In [4]:
# Add time_of_day column
orders_data = (
    orders_data
    .withColumn("time_of_day", 
                F.when((F.hour(F.col("order_date"))>=5)&(F.hour(F.col("order_date"))<12), "morning") \
                .when((F.hour(F.col("order_date"))>=12)&(F.hour(F.col("order_date"))<18), "afternoon") \
                .otherwise("evening"))
    .withColumn("order_date", F.col("order_date").cast("date"))
)

orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day
0,2023-01-22,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,evening
1,2023-01-28,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,afternoon
2,2023-01-17,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,afternoon
3,2023-01-05,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,evening
4,2023-01-25,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,morning


In [5]:
orders_data.select("purchase_address").collect()

[Row(purchase_address='944 Walnut St, Boston, MA 02215'),
 Row(purchase_address='185 Maple St, Portland, OR 97035'),
 Row(purchase_address='538 Adams St, San Francisco, CA 94016'),
 Row(purchase_address='738 10th St, Los Angeles, CA 90001'),
 Row(purchase_address='387 10th St, Austin, TX 73301'),
 Row(purchase_address='775 Willow St, San Francisco, CA 94016'),
 Row(purchase_address='979 Park St, Los Angeles, CA 90001'),
 Row(purchase_address='181 6th St, San Francisco, CA 94016'),
 Row(purchase_address='867 Willow St, Los Angeles, CA 90001'),
 Row(purchase_address='657 Johnson St, San Francisco, CA 94016'),
 Row(purchase_address='492 Walnut St, San Francisco, CA 94016'),
 Row(purchase_address='322 6th St, San Francisco, CA 94016'),
 Row(purchase_address='618 7th St, Los Angeles, CA 90001'),
 Row(purchase_address='512 Wilson St, San Francisco, CA 94016'),
 Row(purchase_address='440 Cedar St, Portland, OR 97035'),
 Row(purchase_address='471 Center St, Los Angeles, CA 90001'),
 Row(purcha

# Transform string columns

In [6]:
orders_data = (
    orders_data
    .withColumn("product", F.lower(F.col("product")))
    .withColumn("category", F.lower(F.col("category")))
    .withColumn("purchase_state", F.regexp_extract(F.col("purchase_address"), r"\b[A-Z]{2}\b", 0))
)

# Reorder columns

In [7]:
order_columns = [
    "order_date",
    "time_of_day",
    "order_id",
    "product",
    "product_id",
    "category",
    "purchase_address",
    "purchase_state",
    "quantity_ordered",
    "price_each",
    "cost_price",
    "turnover",
    "margin"
]
orders_data = orders_data.select(order_columns)

# Write cleaned data

In [8]:
orders_data.write.parquet("orders_data_clean", mode="overwrite")

# Stop Spark session

In [ ]:
spark.stop()